# HySwash: A hybrid method for nearshore wave processes

![sketch](./assets/hyswash-sketch.png)

If you want to run this notebook using the pickle files exported in Part 1, execute the code below. Otherwise, you can skip it.

In [ ]:
import requests
import tarfile
import os

url = "http://personales.unican.es/fernanqv/ChySwash_exported.tar.gz"
response = requests.get(url, stream=True)
file = tarfile.open(fileobj=response.raw, mode="r|gz")
file.extractall(path=os.getcwd())

In [1]:
import os
import os.path as op
import numpy as np
import utils.plotting
import bluemath_tk.topo_bathy.profiles
from bluemath_tk.datamining.lhs import LHS
from bluemath_tk.datamining.mda import MDA
from bluemath_tk.waves.series import waves_dispersion 
from bluemath_tk.wrappers.swash.swash_wrapper import HySwashVeggyModelWrapper
from bluemath_tk.core.io import load_model
import xarray as xr

root_dir = os.getcwd()
#output_dir = "/discos/rapido/outputVeggy"
output_dir = "/lustre/geocean/DATA/hidronas1/valva/Veggy_topo_alba"
templates_dir = os.path.join(root_dir, "templates", "VeggyBig")
export_dir = op.join(root_dir, "HyVeggy_exported")

variables_to_analyse_in_metamodel = ["Hs", "Hs_L0", "WL","vegetation_height","plants_density"]
lhs_parameters = {
    "num_dimensions": 5,
    "num_samples": 10000,
    "dimensions_names": variables_to_analyse_in_metamodel,
    "lower_bounds": [0.5, 0.005, 0, 0, 0],
    "upper_bounds": [2, 0.05, 1, 1.5, 1000],
}

depth = np.loadtxt(os.path.join(templates_dir, "depth.bot"))

Load swash_model and mda from a pickle file and postprocessed_output NetCDF file

In [2]:
mda=load_model(op.join(export_dir, "mda_model.pkl"))
swash_model=load_model(op.join(export_dir, "swash_model.pkl"))
postprocessed_output = xr.open_dataset(op.join(output_dir, "output_postprocessed.nc"))
postprocessed_clean = xr.open_dataset(op.join(output_dir, "output_postprocessed_clean.nc"))

## 5. Reconstruction: Principal Component Analysis (PCA) & Radial Basis Fucntions (RBF)

The reconstruction of the time series of wave parameters in the position of the buoy is carried out by an interpolation technique based on radial basis functions (RBF), a scheme wich is very convenient for scatter and multivariate data. The RBF approximation has been applied successfully in many fields, usually with better results than other interpolation methods (Hardy, 1990).
    
Suppose that $f=f(x)$ is the real-valued function that we want to approximate. We are given M scattered data points $\{x_1,..., x_M\}$ of dimension $\textit{n}$ and the associated real function values $\{f_1, ..., f_M\}$, being $f_i = f(x_j), j = 1,...,M$. The RBF interpolation method consists of a weighted sum of radially symmetric basic functions located at the data points. The approximation function is assumed to be of the form:
$$RBF(x) = p(x) + \sum\limits_{j=1}^M a_j\Phi{\large (}{\large \parallel}{x - x_j}{\large \parallel}{\large )}$$

In [ ]:
from bluemath_tk.datamining.pca import PCA
from bluemath_tk.interpolation.rbf import RBF

postprocessed_output=postprocessed_clean.copy(deep=True)

# Apply PCA to the postprocessed output
pca = PCA()
_pcs_ds = pca.fit_transform(
    data=postprocessed_output,
    vars_to_stack=["Hs"],
    coords_to_stack=["Xp"],
    pca_dim_for_rows="case_num",
    value_to_replace_nans={"Hs": 0.0},
)

# Apply RBF reconstruction
rbf = RBF()
rbf.fit(
    subset_data=mda.centroids.iloc[postprocessed_output["case_num"].values, :],
    target_data=pca.pcs_df,
)

pca.save_model(
    model_path=op.join(export_dir, "pca_model.pkl"),
)
rbf.save_model(
    model_path=op.join(export_dir, "rbf_model.pkl"),
)

In [ ]:
pca=load_model(op.join(export_dir, "pca_model.pkl"))
rbf=load_model(op.join(export_dir, "rbf_model.pkl"))


In [ ]:
from utils.plotting import show_graph_for_different_parameters
show_graph_for_different_parameters(pca=pca, rbf=rbf,lhs_parameters=lhs_parameters)

interactive(children=(FloatSlider(value=1.0618101782710436, description='Hs', max=2.0, min=0.5, step=0.15), Fl…

<function utils.plotting.show_graph_for_different_parameters.<locals>.update_plot(Hs, Hs_L0, WL, vegetation_height, plants_density)>

In [50]:
from utils.plotting import show_graph_for_all_vegetations

show_graph_for_all_vegetations(pca=pca, rbf=rbf, depth=depth, hs=2.0, hs_l0=0.02)

ValueError: operands could not be broadcast together with shapes (100, 1, 3) (1, 819, 5)